<h1><center>Leetcode 1 - 100 </center></h1>

### 1 two sum
- Keep track of a dict [elem] -> [index]

In [ ]:
class Solution(object):
     def twoSum(self, nums, target):
        """
        :type nums: List[int]
        :type target: int
        :rtype: List[int]
        """
        numToIdx = dict()
        for i in range(len(nums)):
            current = nums[i]
            diff = target - current
            if diff in numToIdx:
                j = numToIdx[diff]
                return [i, j]
            else:
                numToIdx[current] = i

### 2 ListNode addition
- Modular: <br>
(a) list_to_num()   
(b) num_to_list()    
(c) list_traversal()

In [ ]:
# Definition for singly-linked list.
class ListNode:
     def __init__(self, val=0, next=None):
         self.val = val
         self.next = next
class Solution:
    def list_to_num(self, node):
        # Base case
        if node is None:
            return 0
        # Recursive case
        head_val = node.val
        return head_val + 10 * self.list_to_num(node.next)
    
    # Requires: num is not originally 0
    def num_to_list(self, num):
        # Base case
        if 0 <= num < 10:
            node = ListNode(num)
            node.next = None
            return node
        # Recursive case
        else:
            head_val = num % 10
            node = ListNode(head_val)
            node.next = self.num_to_list(num // 10)
            return node
            
    def addTwoNumbers(self, l1: Optional[ListNode], l2: Optional[ListNode]) -> Optional[ListNode]:
        num1 = self.list_to_num(l1)
        num2 = self.list_to_num(l2)
        print("num1", num1)
        print("num2", num2)
        res_val = num1 + num2
        res_node = self.num_to_list(res_val)
        return res_node

### 3 Longest Substring Without Repeating Characters
$Sliding Window$
 - vars: `lo, hi, dict`
 - init: `dict = dict; lo = hi = 0`
 - invariant: at the end of each iteration, `[lo, hi)` is always a valid window
```
while hi not reach the end:
    move hi by one, record its impact
    move lo multiple steps until the window is valid again
    update the window length
```

In [ ]:
def sliding_window(s, alphabet):
    # vars
    lo, hi, d = 0, 0, 0
    count = dict()
    # init
    for c in alphabet:
        count[c] = 0
    # traverse invariant: 
    # at the end of each iteration, [lo, hi) is always a valid window
    while hi < len(s):
        # 1. move hi every time by one, record its impact
        right = s[hi]
        count[ right ] += 1
        hi += 1
        # 2. move lo until the window is valid
        while count[right] > 1:
            left = s[lo]
            count[ left ] -= 1
            left += 1
        # 3. update the window length
        d = max(d, right-left)
    return d

In [ ]:
class Solution(object):
    def lengthOfLongestSubstring(self, s):
        """
        :type s: str
        :rtype: int
        """
        # Handle degenerate case
        if len(s) == 0: 
            return 0
        
        # Handle normal case
        maxLen = 0
        current = ""
        for c in s:
            if c not in current:
                current = current + c
            else:
                # ---- c ----- | c
                idx = current.index(c)
                # update maxLen
                maxLen = max(maxLen, len(current))
                # updaet current
                current = current[idx+1:] + c
        return max(maxLen, len(current))                
            

### 4 Median of Two Sorted Arrays
 - Visualize:
 ```
        #  ...... leftMaxX [partitionX] rightMinX .............
        #  ............. leftMaxY [partitionY] rightMinY ......
 ```
 - Binary search target = `partitionX` in range [0, lenX)
 - Avoid index error -> Always binsearch on the smaller array `X`
   - partitionIdx in `X` set to be valid
   - since `lenX` < `lenY`, the calculated partitionIdx in `Y` is also valid
 - 化解问题 Median of two arrays -> Fix one pivot in an array automatically fixes the pivot in the other array

In [ ]:
def findMedianSortedArrays(self, nums1: List[int], nums2: List[int]) -> float:
    if len(nums1) < len(nums2):
        X = nums1
        Y = nums2
    else:
        X = nums2
        Y = nums1
    
    # partitionX = number of X elements in the left part 
    # partitionY = number of Y elements in the left part
    # partitionX + partitionY = (lenX + lenY) + 1 // 2
    
    # range of partitionX = [0, lenX+1)
    lo, hi = 0, len(X)+1
    while lo < hi: # while the range [lo, hi) is not empty
        print('lo', lo, 'hi', hi)
        partitionX = (lo + hi) // 2
        partitionY = (len(X) + len(Y) + 1) // 2 - partitionX

        leftMaxX = -math.inf if partitionX == 0 else X[partitionX-1]
        rightMinX = math.inf if partitionX == len(X) else X[partitionX]
        
        leftMaxY = -math.inf if partitionY == 0 else Y[partitionY-1]
        rightMinY = math.inf if partitionY == len(Y) else Y[partitionY]
        
        # Three case
        #  ...... leftMaxX | rightMinX .............
        #  ............. leftMaxY | rightMinY ......
        if leftMaxX <= rightMinY and leftMaxY <= rightMinX:
            if (len(X) + len(Y)) % 2 == 0:
                return (
                    max(leftMaxX, leftMaxY) 
                    +
                    min(rightMinX, rightMinY)
                ) / 2
            else:
                return max(leftMaxX, leftMaxY)
                
        elif leftMaxX > rightMinY:
            hi = partitionX
        else: # leftMaxY > rightMinX
            lo = partitionX + 1
    
    raise Exception("mid not found")

### 5 Longest Palindromic Substring
- Dynamic programming
- createMatrix = [ [0 for _ in range(col_n)] for _ in range(row_n) ]

In [ ]:
class Solution(object):
def make2dList(self, rowNum, colNum):
    """
    This function creates a rowNum by colNum 2d list
    """
    a = []
    for row in range(rowNum):
        a = a + [[False] * colNum]
    return a

def longestPalindrome(self, s):
    """
    :type s: str
    :rtype: str
    """
    """
    isPalindrome[i][j] := if the substring from i to j is a palindrome
    """
    n = len(s)
    isPalindrome = self.make2dList(n+1, n+1)
    # Base case
    for i in range(n):
        isPalindrome[i][i+1] = True
    for i in range(n-1):
        isPalindrome[i][i+2] = (s[i] == s[i+1])
    # Recursive Case
    for l in range(3, n+1):
        for start in range(n-l+1):
            end = start + l
            isPalindrome[start][end] = s[start] == s[end-1] and isPalindrome[start+1][end-1]
    # Traverse the table from longest to shortest
    for l in range(n, 0, -1):
        for start in range(n-l+1):
            end = start + l
            if isPalindrome[start][end]:
                return s[start:end]
    return ""

def solution2(s):
    chars = list(s)
    char_n = len(chars)
    lst = [] * char_n
    rd = 0
    while rd < char_n:
        lst[2*rd] = s[rd]

    max_len = 0
    for center in range(0, len(lst)):
        l, r = center
        cur_len = 0 if lst[center] == 'a' else 1
        while 0 <= l and r < char_n and lst[l] == lst[r]:
            l -= 1
            r += 1
            cur_len += 1
        max_len = max(max_len, cur_len)
    return max_len
    

### 9 Palindrome Number
 - Reverse string 不能用 `reversed(s)`. Should use `s[::-1]`

In [ ]:
def isPalindrome(self, x: int) -> bool:
    s = str(x)
    return s == s[::-1]

### 10 Regular Expression Matching
 - Dynamic Programmming `Top Down` Approach
 - Subproblem `(i, j) ->` whether `s[i:]` matches `p[j:]`
 - Casing
    - For single letter `a` or `.`: `i, j` -> `i+1, j+1`
    - For wild card `X*`: 
        - Use wild card: `X macthes` and `i, j` -> `i+1, j`
        - Not use wild card: `i, j` -> `i, j+1`

In [ ]:
def isMatch(self, s: str, p: str) -> bool:
    cache = dict()
    
    # returns if s[i:] matches the pattern of p[j:]
    # cursor i in string and cursor j in 
    def DP(i, j):
        # Base case
        if (i, j) in cache: return cache[(i, j)]
        if i == len(s) and j == len(p): return True
        if j == len(p): return False
        # Note: not sure when only i >= len(s)
        
        # Recursive case
        firstMatch = i < len(s) and (s[i] == p[j] or p[j] == '.')
        
        # ---- Has wild card
        if j+1 < len(p) and p[j+1] == '*':
            useWildCard = firstMatch and DP(i+1, j) # add letter
            notUseWildCard = DP(i, j+2) # use empty
            cache[(i, j)] = useWildCard or notUseWildCard
            return cache[(i, j)]
        # ---- Does not have wild card
        else:
            if firstMatch:
                cache[(i, j)] = DP(i+1, j+1)
            else:
                cache[(i, j)] = False
            return cache[(i, j)]
    
    # Driver
    ans = DP(0, 0)
    return ans

### 11 Container With Most Water
- Two pointers <br>
lo = 0    hi = n-1
***
- Untill lo and hi meets each other:
- if H(lo) <= H(hi): $\implies$ lo is going to be the bottle-neck 
   $\implies$  changing hi doesn't change h , but will make w smaller
   $\implies$  move lo++
- if H(lo) > H(hi) $\implies$ same reasoning, move hi--

In [ ]:
class Solution(object):
def maxArea(self, height):
    """
    :type height: List[int]
    :rtype: int
    Two pointer method
    """
    def getArea(leftIdx, rightIdx):
        l = height[leftIdx]
        r = height[rightIdx]
        h = min(l, r)
        w = rightIdx - leftIdx
        area = h * w
        return area
    
    n = len(height)
    left = 0 # inclusive
    right = n-1 # inclusive
    maxSoFar = getArea(left, right)
    while (left < right):
        '''
        height(left) < height(right)
        ==> 修改 right 不会改变 h
        ==> 修改 right 导致 w 变小 

        x o o o o 
        o o o o
        o o o
        o o 
        o
        
        x o o o o 
        - o o o 
        - o o
        - o 
        -

        '''
        if height[left] < height[right]:
            left += 1
        else:
            right -= 1
        newArea = getArea(left, right)
        maxSoFar = max(maxSoFar, newArea)
    return maxSoFar



### 15 3Sum
- Cost $O(n^2)$
- sort the entire array
- outermost loop 'pivot' at each non-redundant element
  -    判断条件 piv == 0 or nums[piv] != nums[piv-1]
- inner loop: call 2sums:
  -  two pointers lo = piv+1    hi = |nums| - 1
  -  move pointers to each other by comparing nums[lo]+nums[hi] against 
- Avoid triplets 
  - Sort the array $\implies$ identical elements are together
  - (Outer loop) To skip identical, adjacent elements $\implies$ `if piv == 0 or nums[piv-1] != nums[piv]:`
  - (Inner loop) To skip identical, adjacent elements $\implies$ 
  ```
  lo += 1; hi i= 1
  while lo < hi and nums[lo] == nums[lo-1]: 
      lo += 1
  ```

In [ ]:
class Solution:
    # need to collect all the 2sum sols
    def solve2Sum(self, nums, piv, acc):
        n = len(nums)
        lo = piv + 1
        hi = n - 1
        target = 0 - nums[piv]
        while (lo < hi):
            curSum = nums[lo] + nums[hi]
            if curSum == target:
                acc.append([nums[piv], nums[lo], nums[hi]])
                lo += 1
                hi -= 1
                while lo < hi and nums[lo] == nums[lo-1]:
                    lo += 1
            elif curSum < target:
                lo += 1
            else:
                hi -= 1
    
    def threeSum(self, nums):
        nums.sort()
        n = len(nums)
        sol = []
        for piv in range(n):
            if piv == 0 or nums[piv-1] != nums[piv]:
                self.solve2Sum(nums, piv, sol)
        return sol


### 16 3Sum Closest
 - $Two pointer$.
 - Outer loop all `n1` -> `O(n)`
 - Inner loop perform a two pointer traversal for `n2, n3` -> `O(2n)`
    - For outer variable `i1`, perform two-pointer as follows:
        ```
        while i2 < i3:
            n2 = nums[i2]
            n3 = nums[i3]

            # -- update the target variable
            if abs(n1 + n2 + n3 - target) < abs(target_best_i1 - target):
                n2_best = n2
                n3_best = n2 
                target_best_i1 = n1 + n2 + n3

            # -- move the pointer
            if n1 + n2 + n3 < target:
                i2 += 1 
            else:
                i3 -= 1 
        ```
 - Total cost `O(n^2)`
 

In [ ]:
def threeSumClosest(self, nums: List[int], target: int) -> int:
        nums.sort()
        n = len(nums)
        target_best = math.inf

        for i1 in range(n-2):
            # -- skip identical starting number n1
            if i1 > 0 and nums[i1-1] == nums[i1]: continue
            n1 = nums[i1]
            
            # -- with n1, find n2 and n3 using two pointers
            i2 = i1 + 1
            i3 = n - 1
            n2_best, n3_best = nums[i2], nums[i3]
            target_best_i1 = n1 + n2_best + n3_best
            
            while i2 < i3:
                n2 = nums[i2]
                n3 = nums[i3]
                # --- no need to approximate
                if n1 + n2 + n3 == target: return target
                if abs(n1 + n2 + n3 - target) < abs(target_best_i1 - target):
                    n2_best = n2
                    n3_best = n2 
                    target_best_i1 = n1 + n2 + n3
                # -- move the pointer
                if n1 + n2 + n3 < target:
                    i2 += 1 
                else:
                    i3 -= 1 
            # finish finding for this i1
            if abs(target_best_i1 - target) < abs(target_best - target):
                target_best = target_best_i1
        
        # End for loop
        return target_best

### 17 Letter Combinations of a Phone Number
- Init initial `prefixes`
- For each current prefix `prefix`
    - For each new letter `c'
        - collect `c + prefix`

In [ ]:
class Solution:
    def letterCombinations(self, digits: str) -> List[str]:
        # Handle degenerate cases
        if len(digits) == 0:
            return []
        
        # Handle normal cases
        digit_to_letters = {
            '2' : ['a', 'b', 'c'],
            '3' : ['d', 'e', 'f'],
            '4' : ['g', 'h', 'i'],
            '5' : ['j', 'k', 'l'],
            '6' : ['m', 'n', 'o'],
            '7' : ['p', 'q', 'r', 's'],
            '8' : ['t', 'u', 'v'],
            '9' : ['w', 'x', 'y', 'z']
        }
        
        prefixes = digit_to_letters[digits[0]]
        for digit in digits[1:]:
            letters = digit_to_letters[digit]
            new_prefixes = []
            for prefix in prefixes:
                for letter in letters:
                    new_prefixes.append(prefix + letter)
            prefixes = new_prefixes
            
        return prefixes

### 18 4Sum
 - General `kSum` problem - Reduce problelm to `2Sum` using two-pointer method.
 - Cost $O(n^{k-1})$

In [ ]:
######################### SOL 1 ##############################
def fourSum___BACKTRACK(self, nums: List[int], target: int) -> List[List[int]]:
    nums.sort()
    n = len(nums)
    quads = []
    
    def backtrack(cursor, quad, quadsum):
        if len(quad) == 4:
            if quadsum == target:
                quads.append(quad.copy())
            return
        for new_cursor in range(cursor, n):
            if new_cursor > cursor and nums[new_cursor] == nums[new_cursor-1]:
                continue
            newnum = nums[new_cursor]
            quad.append(newnum)
            backtrack(new_cursor+1, quad, quadsum+newnum)
            quad.pop()
        
    backtrack(0, [], 0)
    return quads

    ######################### SOL 2 ##############################
def twoSum(self, nums, target): # or use the hashment solution freq[num]->its idx of occurence
    sols = []
    left, right = 0, len(nums)-1
    while left < right:
        curSum = nums[left] + nums[right]
        if curSum == target:
            sols.append([nums[left], nums[right]])
            left += 1
            right -= 1
            while left < right and nums[left] == nums[left-1]: left += 1
        elif curSum < target:
            left += 1
        else:
            right -= 1
    return sols


# This function returns all th e
def kSum(self, k, nums, target):
    # Base case
    if len(nums) == 0:
        return []
    if k == 2:
        return self.twoSum(nums, target)
    # Recursive case
    k_sols = []
    for pivot in range(len(nums)):
        # skip all adjacent identical pivots
        if pivot == 0 or nums[pivot] != nums[pivot-1]:
            k_minus_1_sols = self.kSum(k-1, nums[pivot+1:], target-nums[pivot])
            for L in k_minus_1_sols:
                k_sols.append([nums[pivot]] + L)
    return k_sols

### 19 Remove Nth Node From End of List
* Fast vs Slow nodes
* initially,
<br> Fast = n+1 th
<br>  Slow = 1 th
* at the end,
<br> Fast = m th
<br> Slow = m-n th = last nth

In [ ]:
# Definition for singly-linked list.
class ListNode(object):
     def __init__(self, val=0, next=None):
         self.val = val
         self.next = next
class Solution(object):
    def getLength(self, head):
        # base case
        if head == None:
            return 0
        else:
            return 1 + self.getLength(head.next)
        
    def removeNthFromEnd(self, head, n):
        """
        :type head: ListNode
        :type n: int
        :rtype: ListNode
        !!! Adding a dummy node helps to remove edges cases in the operations of a linked list
        """
        # Handle edge cases
        
        # Two pointers called fast and slow
        # Initialize: let fast advance for n nodes before slow
        dummy = ListNode()
        dummy.next = head
        fast = dummy
        slow = dummy
        for _ in range(n+1): 
            fast = fast.next
        
        # Traverse the linked list
        while (fast != None):
            slow = slow.next
            fast = fast.next
        
        # Now that fast has reached the end
        slow.next = slow.next.next
        return dummy.next

### 20 Valid Parentheses
- For matching parens, "match" problems, use stack (list)
- `stack.pop = list.pop()`
- `stack.push = list.append(elem)`

In [ ]:
class Solution(object):
    def isValid(self, s):
        """
        :type s: str
        :rtype: bool
        """

        stack = [ ]
        opens = "({["
        closes = ")}]"
        match = {'(' : ')',
                 '[' : ']',
                 '{' : '}'}
        
        # Traverse the string of parens
        for paren in s:
            # open parens
            if paren in opens:
                stack.append(paren)

            # close parens
            else:
                if not stack:
                    return False
                else:
                    # stack not empty
                    top = stack.pop()
                    if not (top in opens and match[top] == paren):
                        return False
        return not stack

### 21 Merge Two Sorted Lists
- keep two reader nodes and one writer node

In [ ]:
class Solution(object):
    def mergeTwoLists(self, list1, list2):
        """
        :type list1: Optional[ListNode]
        :type list2: Optional[ListNode]
        :rtype: Optional[ListNode]
        Iterative version
        """
        res = ListNode(-1, None)
        writer = res
        l1 = list1
        l2 = list2
        while l1 and l2:
            if l1.val <= l2.val:
                newNode = ListNode(l1.val, None)
                writer.next = newNode
                writer = writer.next
                l1 = l1.next
            else:
                newNode = ListNode(l2.val, None)
                writer.next = newNode
                writer = writer.next
                l2 = l2.next
        while l1:
            newNode = ListNode(l1.val, None)
            writer.next = newNode
            writer = writer.next
            l1 = l1.next
        while l2:
            newNode = ListNode(l2.val, None)
            writer.next = newNode
            writer = writer.next
            l2 = l2.next
        return res.next

### 22 Generate Parentheses
- Backtracking keep track of `cur_sol`, `open_n` and `close_n`

In [ ]:
class Solution:
    def generateParenthesis(self, n: int) -> List[str]:
        sol = []
        
        # invariant: the current solution has openN ( and closeN )
        # invariant: openN >= closeN 
        def backtrack(cur, openN, closeN):
            # Base case
            if openN + closeN == 2 * n:
                if openN == closeN:
                    sol.append("".join(cur))
                return
            # Recursive case
            # option (1) add an open paren
            if openN < n:
                cur.append('(')
                backtrack(cur, openN+1, closeN)
                cur.pop()
            # option (2) add a close paren
            if closeN < openN:
                cur.append(')')
                backtrack(cur, openN, closeN+1)
                cur.pop()
        
        cur = []
        backtrack(cur, 0, 0)
        return sol

### 23 Merge k Sorted Lists
 - `heapq` all keys have to be distinct ! If not distinct, solution: `heapq.heappush(key, id(node), node)`
    - 通过区分 `id(node)`来differentiate具有相同`val`的不同`node`.
    - 例子 `heapq.heappush(PQ, (node.val, id(node), node))` where `node.val` + `id(node)` is used to break ties and serves as the key in PQ
 - 思路 : a bunch of sorted list -> `Use a priority queue to keep track of the` $\textit{curMin Boundary}$

In [ ]:
import heapq
def mergeKLists(self, lists: List[Optional[ListNode]]) -> Optional[ListNode]:
    PQ = []
    heapq.heapify(PQ)
    
    for l in lists:
        if l:
            heapq.heappush(PQ, (l.val, id(l), l))
    
    dummy = ListNode(-1)
    wr = dummy
    
    while PQ:
        # 1 -- get the current min node
        minVal, minId, minNode = heapq.heappop(PQ)
        
        # 2 -- write the node to the result
        wr.next = ListNode(minVal)
        wr = wr.next
        
        # 3 -- push the next node of this list into heap
        if minNode.next:
            nextNode = minNode.next
            heapq.heappush(PQ, (nextNode.val, id(nextNode), nextNode))
    return dummy.next

### 24 Swap Nodes in Pairs
 - Don't forget to handle degenerate cases firt
 - Use dummy header and return `dummy.next` at last
 - During iteration, keep track of 
    - before `prev -> left -> right -> succ` 
    - after `prev -> right -> left -> succ`
 - Do not forget to use `ptr1 = ptr2.next if ptr2 else None` when it can be `NoneType`

In [ ]:
def swapPairs(self, head: Optional[ListNode]) -> Optional[ListNode]:
    # 1-- Handle degenerate cases
    if not head or not head.next: return head
    
    # 2-- Handle normal cases
    dummy = ListNode(-1, head)
    prev = dummy
    left = dummy.next
    right = dummy.next.next
    succ = dummy.next.next.next
    
    # 3-- Reconnecting each pair
    while right is not None:
        # a-- reconnect pairs
        right.next = left
        left.next = succ
        prev.next = right
        # b-- advance the pointers
        prev = left
        left = succ
        right = succ.next if succ else None
        succ = right.next if right else None
        
    # 4-- Return
    return dummy.next

### 25 Reverse Nodes in k-Group
 - implement helper function to reverse k at a time.
 - calling this on `[]`
 - Step 1: `[] -> 1 -> 2 -> 3 -> 4 -> 5 -> ...` 
 - Step 2: `[] -> 1 <- 2 <- 3 -> 4 -> 5 -> ...` reverse the first k nodes after head
 - Step 3: `[] -> 3 -> 2 -> 1 -> 4 -> 5 -> ...` reconnect the reversed k notes as a whole back to the link
    - and return node `1` for the next reverse
  


In [ ]:
def reverseKOnce(self, head, k):
    # 1-- first check if there are >= k nodes after head
    rightmost = head
    tail = head.next
    
    for _ in range(k):
        rightmost = rightmost.next
        if rightmost is None: return None

    # 2-- reverse the first k nodes after head
    prev, curr = head.next, head.next.next 
    while prev != rightmost:
        succ = curr.next
        curr.next = prev
        prev = curr
        curr = succ
    # 3 -- reconnect the reversed k notes as a whole back to the link
    head.next = rightmost
    tail.next = curr
    return tail

def reverseKGroup(self, head: Optional[ListNode], k: int) -> Optional[ListNode]:
    dummy = ListNode(-1, head)
    cursor = dummy
    
    lastFinishAt = self.reverseKOnce(dummy, k)
    
    while lastFinishAt:
        lastFinishAt = self.reverseKOnce(lastFinishAt, k)
        
    return dummy.next

### 30 Substring with Concatenation of All Words
 - Bundle together `stridle` many characters together $\implies$ Do `stridle`-many sliding windows, each time advance by `stride`
 - Use `collections.Counter()`
 - Observation: each window length `k` is fixed
 - Each iteration
   - examine the new word `curWord`
   - if `curWord` is an alien word empty the entire new window
   - if `curWord` is not alien
     - Add `curWord` to the current window and update correspondingly
     - keep moving `lo` to right until the window is valid again
     - $\blacksquare$ if the window width is exactly `k`, record answers

In [ ]:
def findSubstring(self, s: str, words: List[str]) -> List[int]:
        n = len(s)
        stride = len(words[0])
    
        targetCnt = len(words)
        targetLen =  targetCnt * stride
        targetFreq = collections.Counter(words)
        indices = []
        
        def slidingWindow(lo):
            curCnt = 0
            curFreq = dict()
            
            hi = lo
            while hi + stride <= n:
                # move `hi` to right by one
                curWord = s[hi : hi + stride]
                # ------ case 1: empty the entire window
                if curWord not in targetFreq: 
                    hi = hi + stride
                    lo = hi
                    curFreq = dict()
                    curCnt = 0
                # ------ case 2: a valid word
                else:
                    hi += stride
                    curFreq[curWord] = curFreq.get(curWord, 0) + 1
                    curCnt += 1
                    while hi-lo > targetLen or \
                    curFreq.get(curWord, 0) > targetFreq.get(curWord, 0):
                        loWord = s[lo : lo + stride]
                        curFreq[loWord] -= 1
                        curCnt -= 1
                        lo += stride
                    if curCnt == targetCnt:
                        indices.append(lo)

        # Driver
        for start in range(stride):
            slidingWindow(start)
        return indices

### 32 Longest Valid Parentheses
 - Dynamic Programming `DP[i] = the length of the longest valid sequence ending exactly at [i]`
 - Casing
 - if `s[i] = (` -> has to be 0
 - if `s[i] = )` -> 
    - Case 1
         ```
            (    )
            i-1   i
         ```
    - DP[i] = DP[i-2] + 2
    - Case 2
         ```
       ...   i-DP[i-1]-2    i-DP[i-1]-1                  i-1     i
       ...       |               ?        [     ....      ]      )
                     
         ```
       - if `? = )` -> DP[i] = 0
       - if `? = (` -> DP[i] = $DP[i-DP[i-1]-2] + DP[i-1] + 2$ (from left to right)

In [ ]:
def longestValidParentheses(self, s: str) -> int:
    # Handle degenerate case
    if len(s) == 0: return 0
    
    # Handle normal cases 
    # DP[i] = the length of the longest valid sequence ending exactly at [i]
    n = len(s)
    DP = [0] * n
    
    for i in range(n):
        # -- current char is (
        if s[i] == '(': DP[i] = 0
            
        # -- current char is )
        else:
            if i == 0: DP[i] = 0
            else:
                if s[i-1] == '(' and s[i] == ')':
                    DP[i] = DP[i-2] + 2
                elif s[i-1] == ')' and s[i] == ')':
                    if i-DP[i-1]-1 >= 0 and s[i-DP[i-1]-1] == '(':
                        DP[i] = DP[i-1] + 2 + DP[i-DP[i-1]-2]
                    else:
                        DP[i] = 0
    #print(DP)
    return max(DP)

### 33 Search in Rotated Sorted Array
- first use binsearch to find the rotation index
- compare target with `nums[0]`, we can know which part of the array to search

In [ ]:
class Solution:
# this function uses binary search to find rotation index i 
# such that nums[i] > nums[i+1]
def binsearch_rotIndex(self, nums):
    n = len(nums)
    lo = 0
    hi = n
    while lo < hi:
        # invariant: rotIndex potentially within range [lo, hi)
        mid = lo + (hi-lo) // 2
        # Cases
        # A. mid has no succeeding element to its right
        if mid == n-1:
            if nums[mid-1] > nums[mid]:
                return mid-1
            else:
                return -1
        # B. mid has succeeding element to its right neighbor
        # B(1) mid is not sorted with respect to its right neighbor
        if nums[mid] > nums[mid+1]:
            return mid
        # B(2) mid is sorted with respect to its right neighbor
        else:
            if nums[mid] < nums[0]:
                hi = mid
            else:
                lo = mid + 1
    return -1 # will not reach this line
    
def binsearch(self, nums, target):
    n = len(nums)
    lo = 0
    hi = n
    while (lo < hi):
        mid = lo + (hi-lo) // 2
        if nums[mid] == target:
            return mid
        elif nums[mid] < target:
            lo = mid + 1
        else:
            hi = mid
    return -1 # will not reach this line
    

def search(self, nums: List[int], target: int) -> int:
    if len(nums) == 1:
        if nums[0] == target:
            return 0
        else:
            return -1
    rotIndex = self.binsearch_rotIndex(nums)
    print(rotIndex)
    # case 1: the list is not rotated
    if rotIndex == -1:
        return self.binsearch(nums, target)
    # case 2: if the list is rotated
    # A
    if target == nums[0]:
        return 0
    # B
    elif target < nums[0]:
        res = self.binsearch(nums[rotIndex+1:], target)
        if res != -1:
            return res  + 1 + rotIndex
        else:
            return -1
    # C
    else:
        return self.binsearch(nums[0:rotIndex+1], target)

### 34 Find First and Last Position of Element in Sorted Array
- Sorted array $\implies$ binary search !!!
- 流程 Use two binary searches for the `start` and the `end`. Takes $O(2logn)$ time
- Adaptations:
    - find `start`
    ```
    if nums[mid] == target:
        if mid == 0 or nums[mid-1] < target:
            return mid
        else:
            hi = mid
    ```
    - find `end`
    ```
    if nums[mid] == target:
        if mid == n-1 or nums[mid+1] > target:
            return mid
        else:
            lo = mid + 1
    ```

In [ ]:
class Solution:
    def binsearch_start(self, nums, target):
        n = len(nums)
        lo = 0
        hi = n
        while lo < hi:
            mid = lo + (hi-lo) // 2
            if nums[mid] == target:
                if mid == 0 or nums[mid-1] < target:
                    return mid
                else:
                    hi = mid
            elif nums[mid] < target:
                lo = mid + 1
            else:
                hi = mid
        return -1
    
    def binsearch_end(self, nums, target):
        n = len(nums)
        lo = 0
        hi = n
        while lo < hi:
            mid = lo + (hi-lo) // 2
            if nums[mid] == target:
                if mid == n-1 or nums[mid+1] > target:
                    return mid
                else:
                    lo = mid + 1
            elif nums[mid] < target:
                lo = mid + 1
            else:
                hi = mid
        return -1
    
    def searchRange(self, nums: List[int], target: int) -> List[int]:
        start = self.binsearch_start(nums, target)
        end = self.binsearch_end(nums, target)
        return [start, end]

### 35 Search Insert Position
 - Exactly standard Binary Search
 

In [ ]:
class Solution:
    def binsearch(self, nums, target):
        lo, hi = 0, len(nums)
        while lo < hi:
            mid = lo + (hi-lo) // 2
            if nums[mid] == target:
                return mid
            elif nums[mid] < target:
                lo = mid + 1
            else:
                hi = mid
        return lo
    def searchInsert(self, nums: List[int], target: int) -> int:
        return self.binsearch(nums, target)

### 37 Sudoku Solver
 - __Back Track__

In [ ]:
def solveSudoku(self, board: List[List[str]]) -> None:
    """
    Do not return anything, modify board in-place instead.
    """
    """
    box labelling
    0  1  2 
    3  4  5
    6  7  8
    
    rows 0 1 2 3 4 5 6 7 8
    cols 0 1 2 3 4 5 6 7 8
    """
    def boxIdx(row, col):
        return (row // 3) * 3 + col // 3
    
    def nextPos(r, c):
        if c < 8: return (r, c+1)
        else: return (r+1, 0)
    
    rowContain = []
    colContain = []
    blockContain = []
    
    for i in range(9):
        rowContain.append(set())
        colContain.append(set())
        blockContain.append(set())
    
    for r in range(9):
        for c in range(9):
            num = board[r][c]
            if num != '.':
                rowContain[r].add(num)
                colContain[c].add(num)
                blockContain[boxIdx(r, c)].add(num)
    
    def backtrack(r, c):
        if r > 8:
            return True# success
        elif board[r][c] != '.':
            newR, newC = nextPos(r, c)
            return backtrack(newR, newC)
        else:
            for num in range(1, 10):
                fillin = str(num)
                if fillin in rowContain[r] or \
                    fillin in colContain[c] or \
                    fillin in blockContain[boxIdx(r, c)]:
                    continue
                
                # --- modify
                board[r][c] = fillin
                rowContain[r].add(fillin)
                colContain[c].add(fillin)
                blockContain[boxIdx(r, c)].add(fillin)
                # --- recurse
                newR, newC = nextPos(r, c)
                if backtrack(newR, newC):
                    return True
                # --- restore 
                board[r][c] = '.'
                rowContain[r].remove(fillin)
                colContain[c].remove(fillin)
                blockContain[boxIdx(r, c)].remove(fillin)
            return False
    ## Driver
    backtrack(0, 0)

### 39 Combination Sum
- Backtrack pseudocode <br> 
**cursor is used to avoid duplicat trials**
```
 options = [ ... ]
 acc_sol = [ ]
 def backtrack(cursor, cur_sol, params):
   if found_a_sol:
       acc_sol += cur_sol
   else:
       for new_cursor in range(cursor, ...):
           if  valid:
               apply_change(cur_sol)
               backtrack(new_cursor, cur_sol, params')
               remove_change(cur_sol)    
```
- More about backtracking
    - 使用场景 You are faced with a number of options, and you must choose one of these. After you make your choice you will get a new set of options; just what set of options you get depends on what choice you made. This procedure is repeated over and over until you reach a final state. If you made a good sequence of choices, your final state is a goal state; if you didn't, it isn't.
    - Underlying decision tree (for yes/no problems)
        ```
        solve(Node n) 
            if n is a leaf node:
                if the leaf is a goal node: return true
                else return false

            else:
                for each child c of n 
                    if solve(c) succeeds, return true
            
            return false
        ```
    - Underlying tree (for general problems)
    ```
    sols = []
    solve(n):
        if n is a leaf node:
            if the leaf node is a good sol:
                sols += n
            return 
        else:
            for each child c of n
                solve(c)
    ```
    - Using stack instead of recursion
    ```
    solve(Node n):
        put node n on the stack;
        while the stack is not empty:
            if the node at the top of the stack is a leaf:
                if it is a goal node, return true
                else pop it off the stack
            
            else:
                if the node at the top of the stack has untried children
                    push the next untried child onto the stack
                else pop the node off the stack
        return false
    ```

In [ ]:
class Solution:
    def copyList(self, l):
        copy = []
        for ele in l:
            copy.append(ele)
        return copy
        
    def combinationSum(self, candidates: List[int], target: int) -> List[List[int]]:
        n = len(candidates)
        combs = []
        
        def backtrack(curr_comb, cursor, remain):
            # Base case
            if remain == 0:
                sol = self.copyList(curr_comb)
                combs.append(sol)
                return
            if remain < 0:
                return 
            # Recursive Case
            for new_cursor in range(cursor, n):
                # apply change
                curr_comb.append(candidates[new_cursor])
                # recurse
                backtrack(curr_comb, new_cursor, remain-candidates[new_cursor])
                # remove change
                curr_comb.pop()
            return
        
        # Main
        backtrack([], 0, target)
        return combs

### 39 Combination Sum
- Transpose + reverse each row

In [ ]:
class Solution(object):
    def diagonalFlip(self, matrix):
        n = len(matrix)
        for i in range(n-1):
            for j in range(n-1-i):
                ii = n-1-j
                jj = n-1-i
                tmp = matrix[i][j]
                matrix[i][j] = matrix[ii][jj]
                matrix[ii][jj] = tmp
    
    def flipRow(self, matrix, r1, r2):
        n = len(matrix)
        for c in range(n):
            tmp = matrix[r1][c]
            matrix[r1][c] = matrix[r2][c]
            matrix[r2][c] = tmp
    
    def upsideDownFlip(self, matrix):
        n = len(matrix)
        for r in range(n//2):
            rr = n-1-r
            self.flipRow(matrix, r, rr)
    
    def rotate(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: None Do not return anything, modify matrix in-place instead.
        """
        self.diagonalFlip(matrix)
        self.upsideDownFlip(matrix)

### 40 Combination Sum II
 - Backtrack
 - `cursor`: in order to grow the solution in this current round, where can we start picking elements from
   - always: `for new_cursor in range(cursor, n)`
   - not allow duplicate choices $\implies$ `new_cursor`
   - allow duplicate choices $\implies$ `new_cursor + 1`
 - Early stopping
 - Prevent duplicate combiniations:
 ```
 e.g.      2    2    2    4    5    7    7    8
           *              *    *    *         *
 ```

In [ ]:
class Solution:
    def combinationSum2(self, candidates: List[int], target: int) -> List[List[int]]:
        n = len(candidates)
        combs = []
        candidates.sort()
        
        def backtrack(cursor, sum_so_far, comb_so_far):
            if sum_so_far == target:
                combs.append(comb_so_far.copy())
                return
            if sum_so_far > target:
                return
            else:
                for new_cursor in range(cursor, n):
                    new_num = candidates[new_cursor]
                    
                    # avoid duplicate combs
                    if new_cursor != cursor and new_num == candidates[new_cursor-1]:
                        continue
                    
                    comb_so_far.append(new_num)
                    backtrack(new_cursor+1, sum_so_far+new_num, comb_so_far)
                    comb_so_far.pop()
                    
        backtrack(0, 0, [])
        return combs

### 42 Trapping Rain Water
 - Scan 方法: scan to get `highestSoFarFromLeft` and `highestSoFarFromRight` -> take `max(0, min(left, right))` for each vertical bar
 - One Pass Stack 方法: 
    - invariant `in stack: stores the index of buildings such that each building is strictly lower than the one below it`.
    - Each time keep track of `left, mid, right`
    - Each time add extra water = `(right - left - 1)` * `min(leftH, rightH) - midH` into the total 
 - Two Pointer 方法:  
    - 思路: (two pass 简化成 two pointer) -> __if there is a larger bar at one end (say right), we are assured that the water trapped would be dependant on height of bar in current direction (from left to right)__
    - `left` starting from 0, `right` starting from n-1. `left` and `right` corresponds to the vertical base upon which we add water each iteration.
    - See code comment

In [ ]:
##################### 方法 1: Scan ######################
def trap(self, height: List[int]) -> int:
    # Handle degenerate case
    if len(height) <= 2: return 0
    
    # Handle normal case
    n = len(height)
    highestSoFarFromLeft = []
    highestSoFarFromRight = []
    
    maxSoFar = 0
    for h in height:
        highestSoFarFromLeft.append(maxSoFar)
        maxSoFar = max(maxSoFar, h)
    
    maxSoFar = 0
    for h in height[::-1]:
        highestSoFarFromRight.append(maxSoFar)
        maxSoFar = max(maxSoFar, h)
    
    highestSoFarFromRight.reverse()
    waterHeight = []
    for i, (leftH, rightH) in \
    enumerate(zip(highestSoFarFromLeft, highestSoFarFromRight)):
        waterHeight.append(max( 0, 
                                min(leftH, rightH) - height[i]))
    
    return sum(waterHeight)

##################### 方法2: one pass, nested loop, scan ######################
def trap(self, height: List[int]) -> int:
    n = len(height)
    stack = [] # a cursor in the buildings
    right = 0 # invariant: buildings in non-ascending order
    water = 0
    
    while right < n:
        while stack and height[right] > height[stack[-1]]:
            mid = stack[-1]
            stack.pop()
            if not stack: break
            left = stack[-1]
            
            # now we have leftH, midH, and rightH
            leftH = height[left]
            midH = height[mid]
            rightH = height[right]
            
            dist = right - left - 1
            waterIncrease = dist * (  min(leftH, rightH) - midH  )
            water += waterIncrease
        
        # finished collecting all water hold by 'right' building
        stack.append(right)
        right += 1
    
    return water

##################### 方法3: Two pointer ######################
def trap(self, height: List[int]) -> int:
    n = len(height)
    left, right = 0, n-1
    leftMax, rightMax = 0, 0
    waterTotal = 0
    
    while left <= right: # !!! <=
        #print(left, right)
        # inv here: maxFromLeft is max of height[:left]
        # inv here: maxFromRight is max of height[right+1:]
        leftH, rightH = height[left], height[right]
        
        # --- Case A: left bar bound by 'from left' direction
        if leftMax < rightMax:
            waterTotal += max(0, leftMax - leftH)
            leftMax = max(leftMax, leftH)
            left += 1
            
        # --- Case B: this bar bound by 'from right' direction
        else:
            waterTotal += max(0, rightMax - rightH)
            rightMax = max(rightMax, rightH)
            right -= 1

    return waterTotal

### 44 Wildcard Matching
 - The same as 10 regex matching
 - Base case list out all 4 clearly
 - There are at most $|p|$ problem $(|s|, ? )$ costing $O(|p|)$. The other $|s|*|p|$ problems costing $O(1)$.
 - Easily optimized to strictly $|p||s|$

In [ ]:
def isMatch(self, s: str, p: str) -> bool:
    cache = dict()
    
    # tests is s[i:] matches the pattern of p[j:]
    def DP(i, j):
        ### Base Cases
        if (i, j) in cache: return cache[(i,j)]
        if i >= len(s) and j >= len(p): return True
        if i >= len(s) and j < len(p):
            for c in p[j:]:
                if c != '*':
                    cache[(i, j)] = False
                    return False
            cache[(i, j)] = True
            return True
        if i < len(s) and j >= len(p): return False
        
        ### Recursive Cases
        firstMatch = i < len(s) and (s[i] == p[j] or p[j] == '?' or p[j] == '*')
        # ---- Has star
        if p[j] == '*':
            # -- choice 1: apply wild card to match a char in s
            useStar = DP(i+1, j)
            # -- choice 2: do not use wild card
            notUseStar = DP(i, j+1)
            # --
            cache[(i, j)] = useStar or notUseStar
            return cache[(i, j)]
        # ---- Does not have star
        else:
            if firstMatch:
                cache[(i, j)] = DP(i+1, j+1)
            else:
                cache[(i, j)] = False
            return cache[(i, j)]
    
    return DP(0, 0)

### 45 Jump Game II

### 46 Permutations

In [ ]:
class Solution:
    def interleave(self, ele, lst):
        n = len(lst)
        res = []
        for i in range(n+1):
            newl = lst[0:i] + [ele] + lst[i:]
            res.append(newl)
        return res
    
    def permute(self, nums: List[int]) -> List[List[int]]:
        # base case
        if len(nums) == 0:
            return []
        if len(nums) == 1:
            return [nums]

        # Recursive case
        rec = self.permute(nums[1:])
        res = []
        for l in rec:
            res = res + self.interleave(nums[0], l)
        return res

### 47 Permutations II
 - Backtrack
 - Viewing each _unique_ number as a choice
 - Use a counter to keep track of how many instances of a _unique_ number has been used
 - `Counter()` returns a mapping of each hashabel element in an iterable to its occurence
 - Prevent duplicate:
   - prevent overuse of a unique a superset element $\implies$ checks `counter[num] > 0`
   - prevent duplication in-between different `perm`s generated $\implies$ Each time iterate each _unique_ number as a choice <br>
    via `for num in counter.keys()`

In [ ]:
from collections import defaultdict
class Solution:
    def permuteUnique(self, nums: List[int]) -> List[List[int]]:
        n = len(nums)
        perms = []
        
        def backtrack(perm, remain):
            if len(perm) == n:
                print(perm, n)
                perms.append(perm.copy())
                return
            
            for num in remain.keys():
                if remain[num] > 0:
                    perm.append(num)
                    remain[num] -= 1
                    backtrack(perm, remain)
                    remain[num] += 1
                    perm.pop()
                    
            
        remain = defaultdict(int)
        for num in nums:
            remain[num] += 1
            
        backtrack([],remain)
        return perms

### 49 Group Anagrams
- Use `dict` dictionary to collect answers when there is a `hashing` theme
String functions <br>
(1) 判断
```
s.isalpha()
s.isalnum()
s.islower()
s.isupper()
s.isdigit()
```
(2) 转换
```
s.lower()
s.upper()
s.replace(s1, s2)
s.strip()
```
(3) 查找
```
s.count()
s.startswith()
s.endswith()
s.find()
```
Set functions <br>
```
s = set(list)
s.add(elem)
x in s
```
Dict functions <br>
```
d = dict() = dict(l) = {_ : _, _ : _}
len(d)
d[k] = v
del d[k]
key in d
for (key, value) in d.items()
for key in d.keys()
for val in d.values()
```

In [ ]:
class Solution(object):
    def isAnagram(self, s1, s2):
        for c in string.ascii_lowercase:
            if s1.count(c) != s2.count(c):
                return False
        return True
    
    def letterToNumber(self, letter):
        return ord(letter) - ord('a')
    
    def hashWord(self, str):
        primes = [2, 3, 5, 7, 11, 
                  13, 17, 19, 23, 29, 
                  31, 37, 41, 43, 47, 
                  53, 59, 61, 67, 71, 
                  73, 79, 83, 89, 97, 
                  101]
        res = 1
        for c in str:
            res = res * primes[self.letterToNumber(c)]
        return res

    def groupAnagrams(self, strs):
        """
        :type strs: List[str]
        :rtype: List[List[str]]
        """
        d = dict()
        for s in strs:
            key = self.hashWord(s)
            d[key] = d.get(key, []) + [s]
        return d.values()


### 51 N-Queens

In [ ]:
def solveNQueens(self, n: int) -> List[List[str]]:
    boards = []
    
    colAttacked = set()
    addDiagAttacked = set()
    minusDiagAttacked = set()
    
    '''
    Diagonal Labelling via (r + c)
    0  1  2  3
    1  2  3  4 
    2  3  4  5 
    3  4  5  6
    '''
    def backtrack(r, board):
        if r >= n:
            copyBoard = []
            for row in board:
                copyBoard.append(''.join(row))
            boards.append(copyBoard)
        else:
            for c in range(n):
                isSafe = c not in colAttacked and  r+c not in addDiagAttacked and r-c not in minusDiagAttacked
                if isSafe:
                    # --- apply change
                    board[r][c] = 'Q'
                    colAttacked.add(c)
                    addDiagAttacked.add(r+c)
                    minusDiagAttacked.add(r-c)
                    # --- recurse
                    backtrack(r+1, board)
                    # --- go back
                    board[r][c] = '.'
                    colAttacked.remove(c)
                    addDiagAttacked.remove(r+c)
                    minusDiagAttacked.remove(r-c)
    # Driver
    board = self.createMatrix(n, n, '.')
    backtrack(0, board)
    return boards

### 52 N-Queens II

In [ ]:
''' Same as 51 '''

### 53 Maximum Subarray
- Dynamic programming 
- Cadane's algorithm

In [ ]:
class Solution(object):
    def maxSubArray(self, nums):
        """
        :type nums: List[int]
        :rtype: int
        """
        '''
        DP[i] = the maximum contiguous subsequence sum of ending exactly at index i
        '''
        # vars
        DP = [0] * len(nums)
        DP[0] = nums[0] 
        MCSS = DP[0]
        empty = 0 # in other definitions empty = - inf

        # traverse
        for i in range(1, len(nums)):
            DP[i] = max(empty, nums[i], nums[i] + DP[i-1])
        
        # extract answers
        for n in DP:
            MCSS = max(MCSS, n)
        
        return MCSS

### 54 Spiral Matrix

In [ ]:
class Solution(object):
    
    def makeMatrix(self, rows, cols, init):
        return [ [init for _ in range(cols)] for _ in range(rows) ]
    
    # return the next pos by moving along `curDir` from `curPos`
    def nextPos(self, curPos, curDir):
        row = curPos[0]
        col = curPos[1]
        
        if curDir == 'R':
            col += 1
        elif curDir == 'L':
            col -= 1
        elif curDir == 'U':
            row -= 1
        elif curDir == 'D':
            row += 1
        return (row, col)
    
    # destructively mutate `curPos` by moving it along `curDir`
    def movePos(self, curPos, direct):
        if direct == 'R':
            curPos[1] += 1
        elif direct == 'L':
            curPos[1] -= 1
        elif direct == 'U':
            curPos[0] -= 1
        elif direct == 'D':
            curPos[0] += 1
    
    def spiralOrder(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: List[int]
        """
        R = 0
        C = 1
        nextDir = {'R' : 'D', 
                   'D' : 'L',
                   'L' : 'U',
                   'U' : 'R'}
        pos = [0, 0]
        direct = 'R'
        rows = len(matrix)
        cols = len(matrix[0])
        trace = []
        visited = self.makeMatrix(rows, cols, False)
        for step in range(rows * cols):
            # append the current position to trace
            trace.append(matrix[pos[R]][pos[C]])
            visited[pos[R]][pos[C]] = True
            
            # check if need to steer direction
            (newRow, newCol) = self.nextPos(pos, direct) # hypothetical next pos
            if newRow < 0 or newRow >= rows or newCol < 0 or newCol >= cols or visited[newRow][newCol]:
                direct = nextDir[direct]

            # make the move
            self.movePos(pos, direct)
        
        return trace
        

### 55 Jump Game
 - sol 1: memoization with recursion `O(n^2)`
 - sol 2: bottom up `O(n^2)`
 ```
for (int i = nums.length - 2; i >= 0; i--) {
        int furthestJump = Math.min(i + nums[i], nums.length - 1);
        for (int j = i + 1; j <= furthestJump; j++) {
            if (memo[j] == Index.GOOD) {
                memo[i] = Index.GOOD;
                break;
            }
        }
    }
 ```
 - sol 3: greedy `O(n)` $ReverseDirection$ Starting from `n-1` and keep going right
    - Iterate over all `i, dist`s and keep updating the `furtherest` variable. Terminate whenever there is an intermediate gap.

In [ ]:
####### Sol 1 Recursion + Memoization ######
def canJump(self, nums: List[int]) -> bool:
    cache = dict()
    n = len(nums)
    
    # checks if can reach end beginning from index i
    def DP(i):
        #### Base Case
        if i in cache:
            return cache[i]
        if i == len(nums) - 1:
            cache[i] = True
            return True
        if i > len(nums) or i < 0:
            cache[i] = False
            return False
        
        #### Recursive Case
        else:
            farIdx = min(n-1, i + nums[i])
            recs = False
            for nexti in range(i+1, farIdx+1):
                recs = recs or DP(nexti)
            cache[i] = recs
            return recs
    
    res = DP(0)
    #print(cache)
    return res

###### Sol 3 Greedy ######
def canJump(self, nums: List[int]) -> bool:
    furtherest = 0
    
    for i, dist in enumerate(nums):
        if i > furtherest: return False
        else: furtherest = max(furtherest, i + dist)
    
    return furtherest >= len(nums) - 1

### 56 Merge Intervals
- 常用 interval 题目技巧: sort according to first field

In [ ]:
class Solution:
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        # S for start, E for end
        S = 0
        E = 1
        n = len(intervals)

        # Handle degenerate cases
        if n == 0:
            return []
        if n == 1:
            return intervals
            
        # Handle normal cases
        intervals.sort()
        res = []
        i = 0
        while i < n:
            start = intervals[i][S]
            end = intervals[i][E]
            while i < n and intervals[i][S] <= end:
                end = max(end, intervals[i][E])
                i += 1
            res.append([start, end])
        return res

### 57 Insert Interval
- merge interval 一般采用 `min(start1, start2)` 和 `max(end1, end2)`
- 常用多个interval的merge
```
all_start = min(start, start[i])
while i < n and itv[i] overlaps with new_itv:
    all_end = max(all_end, end, ends[i])
```

In [ ]:
class Solution:
    def overlap(self, s1, e1, s2, e2):
        # not overlap means e1 < s2 or e2 < s1
        return e1 >= s2 and e2 >= s1
    
    def interval_overlap(self, itv1, itv2):
        return self.overlap(itv1[0], itv1[1], itv2[0], itv2[1])
    
    def insert(self, intervals: List[List[int]], newInterval: List[int]) -> List[List[int]]:
        # handle degenerate case
        if len(intervals) == 0:
            return [newInterval]
        if newInterval[1] < intervals[0][0]:
            return [newInterval] + intervals
        if newInterval[0] > intervals[len(intervals)-1][1]:
            return intervals + [newInterval]
        
        # look for the first interval that overlaps with newInterval
        S = 0
        E = 1
        n = len(intervals)
        i = 0
        while i < n:
            if self.overlap(intervals[i][S], intervals[i][E], newInterval[S], newInterval[E]):
                break
            i += 1
        print("i", i)
        # Case 1: the new interval does not cause overlapping
        if i == n:
            for j in range(n-1):
                s1 = intervals[j][S]
                e1 = intervals[j][E]
                s2 = intervals[j+1][S]
                e2 = intervals[j+1][E]
                # new itv in between jth and j+1th old intervals
                if e1 < newInterval[0] and newInterval[1] < s2:
                    print("here")
                    return intervals[:j+1] + [newInterval] + intervals[j+1:]
        # Case 2
        # intervals[i] is the first one that overlaps with the new interval
        else:
            res = intervals[0:i]
            allStart = min(newInterval[S], intervals[i][S])
            allEnd = 0
            while i < n and self.interval_overlap(newInterval, intervals[i]):
                allEnd = max(allEnd, newInterval[E], intervals[i][E])
                i += 1
            # now allStart, allEnd is finished
            # now i points to the first interval not overlapped by the new interval
            res.append([allStart, allEnd])
            res = res + intervals[i:]
        
            return res

### 58 Length of Last Word

In [ ]:
''' Trivial '''

### 59 Spiral Matrix II
 - Matrix spiral useful functions
    - `(r', c') = def moveInDirect(direc, r, c)`
    - `(r', c', direc') = nextState(direc, r, c)`

In [ ]:
def generateMatrix(self, n: int) -> List[List[int]]:
    def moveInDirect(direc, r, c):
        if direc == 'L':
            return (r, c-1)
        elif direc == 'R':
            return (r, c+1)
        elif direc == 'U':
            return (r-1, c)
        else:
            return (r+1, c)
    
    nextDirect = {
        'L': 'U',
        'R': 'D',
        'U': 'R',
        'D': 'L'
    }
    
    def nextState(direc, r, c):
        r_, c_ = moveInDirect(direc, r, c)
        if 0 <= r_ < n and 0 <= c_ < n and M[r_][c_] == 0:
            return (direc, r_, c_)
        else:
            direc_ = nextDirect[direc]
            r_, c_ = moveInDirect(direc_, r, c)
            return (direc_, r_, c_)
    
    M = self.createMatrix(n, n, 0)
    direc = 'R'
    r, c = 0, 0
    num = 1
    
    for _ in range(n**2):
        # print(r, c, direc)
        M[r][c] = num
        # -- to the next movement
        num += 1
        direc, r, c = nextState(direc, r, c)
    
    return M

### 61 Rotate List
 - When rotate :想到 `rotate 次数` mod `linkc 长度`
 - Edge case 
    - `head is None or head.next is None`
    - `n % k == 0`

In [ ]:
    
def rotateRight(self, head: Optional[ListNode], k: int) -> Optional[ListNode]:
    n = self.linkLen(head)
    
    # 0 -- handle degenerate case
    if n <= 1:
        return head
    
    # 1 -- find the kth node from end
    k = k % n
    lastK, lastK_prev = head, None
    lastOne, lastOne_prev = head, None
    
    for _ in range(k):
        lastOne_prev = lastOne
        lastOne = lastOne.next
        
    while lastOne:
        lastOne_prev = lastOne
        lastOne = lastOne.next
        lastK_prev = lastK
        lastK = lastK.next
    
    lastOne_prev.next = head
    lastK_prev.next = None
        
    return lastK

### 62 Unique Paths

In [ ]:
class Solution(object):
    def makeMatrix(self, rows, cols, init):
        matrix = []
        for row in range(rows):
            matrix = matrix + [[init] * cols]
        return matrix
    
    def uniquePaths(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        
        DP[i][j] = # of unique paths to go from [0][0] to [i][j]
        """
        DP = self.makeMatrix(m, n, 1)
        
        # Base Case
        for col in range(n):
            DP[0][col] = 1
        for row in range(m):
            DP[row][0] = 1
        
        # Recursive Case
        for row in range(1, m):
            for col in range(1, n):
                DP[row][col] = DP[row-1][col] + DP[row][col-1]
        
        return DP[m-1][n-1]

### 63 Unique Paths II
 - Use O(1) space  -> modifying on the existing matrix
 - Simple `DP[i][j] = DP[i-1][j] + DP[i][j-1]`

In [ ]:
def uniquePathsWithObstacles(self, obstacleGrid: List[List[int]]) -> int:
    # modifying existing array to save space
    M = obstacleGrid
    rowN = len(M)
    colN = len(M[0])
    
    if M[0][0] == 1: return 0
    
    M[0][0] = 1
    for r in range(1, rowN): 
        M[r][0] = 1 if M[r][0] == 0 and M[r-1][0] == 1 else 0
        
    for c in range(1, colN): 
        M[0][c] = 1 if M[0][c] == 0 and M[0][c-1] == 1 else 0
    
    for r in range(1, rowN):
        for c in range(1, colN):
            if M[r][c] == 0:
                M[r][c] = M[r-1][c] + M[r][c-1]
            else:
                M[r][c] = 0

    return M[rowN-1][colN-1]

### 64 Minimum Path Sum
- Dynamic Programming (not graph traversal !!!)
- Can modify in-place
$$grid[r][c] = grid[r][c] + min(grid[r-1][c], grid[r][c-1])$$

In [ ]:
def minPathSum(self, grid: List[List[int]]) -> int:
    if len(grid) == 0 or len(grid[0]) == 0: return 0
    
    R = len(grid)
    C = len(grid[0])
    
    for r in range(1, R): grid[r][0] = grid[r][0] + grid[r-1][0]
    for c in range(1, C): grid[0][c] = grid[0][c] + grid[0][c-1]
    
    for r in range(1, R):
        for c in range(1, C):
            grid[r][c] = grid[r][c] + min(grid[r-1][c], grid[r][c-1])
    
    print(grid)
    return grid[R-1][C-1]

### 67 Add Binary
 - Cannot modify a string via indexing!
 - Join operation `"".join(list)`

In [ ]:
def addBinary(self, a: str, b: str) -> str:
    if len(a) > len(b):
        b = '0' * (len(a) - len(b)) + b
    else:
        a = '0' * (len(b) - len(a)) + a
    
    result = []
    carry = 0
    for i in range(len(a)-1, -1, -1):
        na = int(a[i])
        nb = int(b[i])
        print(na, nb)
        digit = str((na + nb + carry) % 2)
        result.append(digit)
        # -- update the carry
        if na + nb + carry < 2:
            carry = 0
        else:
            carry = 1
    if carry == 1: result.append('1')
    result.reverse()
    return "".join(result)

### 69 Sqrt(x)
 - Binary search $O(lgN) = O(len(x))$
 - Binary seach by comparing `mid^2` vs `target`

In [ ]:
def binsearch(self, x):
    lo = 0
    hi = x + 1
    while lo < hi:
        mid = lo + (hi-lo) // 2
        if mid * mid <= x < (mid+1) * (mid+1):
            return mid
        elif mid * mid > x:
            hi = mid
        else:
            lo = mid + 1
    return -1

def mySqrt(self, x: int) -> int:
    return self.binsearch(x)

### 70 Climbing Stairs

In [ ]:
class Solution(object):
    def climbStairs(self, n):
        """
        :type n: int
        :rtype: int
        DP[i] := number of ways to climb i steps
        """
        # Handling trivial cases
        if n == 1:
            return 1
        if n == 2:
            return 2
        
        # Handling normal cases
        DP = [0] * (n+1)
        
        # Base Case
        DP[1] = 1
        DP[2] = 2
        
        # Recursive Case
        for i in range(3, n+1):
            DP[i] = DP[i-1] + DP[i-2]
        
        return DP[n]

### 72 Edit Distance 
 - MED (minimal edit distance)
 - bottom up dynamic programming

In [ ]:
def createMatrix(self, rows, cols, init):
    return [ [init for _ in range(cols)] for _ in range(rows) ]

def minDistance(self, word1: str, word2: str) -> int:
    len1 = len(word1)
    len2 = len(word2)
    MD = self.createMatrix(len1+1, len2+1, math.inf)
    # --- MD[i,j] = min edit distance of word[:i] and word[2:]
    
    # -- Base case
    MD[0][0] = 0
    for i in range(len1+1): MD[i][0] = i
    for j in range(len2+1): MD[0][j] = j
    
    # -- Recursive case
    for i in range(1, len1+1):
        for j in range(1, len2+1):
            if word1[i-1] == word2[j-1]: MD[i][j] = MD[i-1][j-1]
            else: MD[i][j] = 1 + min(MD[i-1][j], MD[i][j-1], MD[i-1][j-1])
    
    return MD[len1][len2]

### 73 Set Matrix Zeros
-  Takeaway: matrix values can be conteminated appropriately

In [ ]:
class Solution(object):
    def setZeroes(self, matrix):
        """
        :type matrix: List[List[int]]
        :rtype: None Do not return anything, modify matrix in-place instead.
        Space O(1) solution
        """
        rowN = len(matrix)
        colN = len(matrix[0])
        # Want  to use matrix[_][0] and matrix[0][_] to denote if the current
        # row or col needs to be zeroed
        # so first row and first column of matrix is conteminated and needs
        # two additional variables to denote if they need to be zeroed or not
        firstRow = False # first row, first col 本身是否为 0
        firstCol = False
        
        for r in range(rowN):
            for c in range(colN):
                if matrix[r][c] == 0:
                    if r == 0:
                        firstRow = True
                    if c == 0: 
                        firstCol = True
                    matrix[r][0] = 0
                    matrix[0][c] = 0

        for r in range(1, rowN):
            for c in range(1, colN):
                if matrix[r][0] == 0 or matrix[0][c] == 0:
                    matrix[r][c] = 0
        if firstRow:           
            for c in range(colN):
                matrix[0][c] = 0
        if firstCol:
            for r in range(rowN):
                matrix[r][0] = 0
            
        """
        Space O(m+n) solution
        rows = []
        cols = []
        m = len(matrix)
        n = len(matrix[0])
        
        for r in range(m):
            for c in range(n):
                if matrix[r][c] == 0:
                    rows.append(r)
                    cols.append(c)
        
        for row in rows:
            for col in range(n):
                matrix[row][col] = 0
                
        for col in cols:
            for row in range(m):
                matrix[row][col] = 0
        """
     
        

### 74 Search a 2D Matrix
- Reduce a matrix problem to a linear sorted list
- Use the basic binary search
- Strategy 1 - Reduce 2D matrix to a linear list
- Strategy 2 - Sortedness $\implies$ think about binary search

In [ ]:
class Solution:
    def searchMatrix(self, matrix: List[List[int]], target: int) -> bool:
        # Reduce the problem into a linear sorted list
        row_n = len(matrix)
        col_n = len(matrix[0])
        
        # a function that returns (r, c) corresponding to idx
        def idx_to_rc(idx):
            row = idx // col_n
            col = idx % col_n
            return (row, col)
        
        # start binary search
        lo, hi = 0, row_n*col_n
        while lo < hi:
            mid = lo + (hi-lo) // 2
            (r, c) = idx_to_rc(mid)
            elem = matrix[r][c]
            
            if elem == target:
                return True
            elif elem < target:
                lo = mid + 1
            else:
                hi = mid
        return False
            

### 75 Sort Colors
- Two pointer one for `0` and one for `2`
- Pseudocode
```
while p0 < p2:
    cur = L[p]
    (1) 
    if L[p] == 1:
        p += 1
    (2)
    elif L[p] == 0:
        swap(p, p0)
        p0 += 1
        p += 1
    else"
        swap(p, p2)
        p2 -= 1
```

In [ ]:
class Solution:
    def sortColors(self, nums: List[int]) -> None:
        """
        Do not return anything, modify nums in-place instead.
        """
        def swap(i, j):
            tmp = nums[i]
            nums[i] = nums[j]
            nums[j] = tmp
        
        # p0: the right boundary of the segment of 0's
        # p2: the left boundary of the segment of 2's
        # p: the scanner traversing the array
        n = len(nums)
        p0 = 0
        p2 = n - 1
        p = 0
        
        while p <= p2:
            # invariant nums[<p0] = 0
            # invariant nums[>p2] = 2
            
            if nums[p] == 1:
                p += 1
            elif nums[p] == 0:
                swap(p, p0)
                # if p == p0: then we have to advance both p and p0
                # if p > p0, then nums[p0] can only be 1, in this case it does not 
                # matter if we advance p or not
                # hence we unify it by advancing p
                p0 += 1
                p += 1
                # nums[p0] can be 0 or 1
            elif nums[p] == 2:
                swap(p, p2)
                p2 -= 1 
                # p2 is unprobed region, so p does not advance forward
                # nums[p2] can be 0 or 1 or 2

### 76 Minimum Window Substring
 - keep track of `missing` = the number of symbols that are not sufficient in the current window
 - keep track of `shortage[c]` = number of `c` symbols needed in order to make this window valid
 - Every iteration moves `end` to the right by one
 - Every iteration moves `begin` to the right, keep updating the change to `missing` and `shortage[]` and keep 
 updating the change to `width` and `answer` along the way until the window is no longer valid
 - For letters not in 't', `stillNeed[c]` first becomes `-1` and then becomes `0`. 无影响。


In [ ]:
from collections import defaultdict
import math

class Solution:
    def minWindow(self, s: str, t: str) -> str:
        # Sliding window variables
        begin, end, width, ans = 0, 0, math.inf, ""
        stillNeed = defaultdict(int)
        for char in t:
            stillNeed[char] = stillNeed[char] + 1 
            # Invariant: stillNeed[c] = how many `c` still needed
        counter = len(stillNeed)
        
        # slide the window
        while end < len(s):
            # move the `end` by one by taking endChar into the current window
            # and update correspondubfkt
            endChar = s[end]
            stillNeed[endChar] -= 1
            if stillNeed[endChar] == 0:
                counter -= 1
            end += 1
            
            # keep moving `begin` to trim the window
            while counter == 0:
                # ------ suppose [begin, end) is a valid window ------ #
                # conditionally update the answer
                if end-begin < width:    
                    ans = s[begin:end]
                    width = end-begin
                beginChar = s[begin]
                stillNeed[beginChar] += 1
                if stillNeed[beginChar] > 0:
                    counter += 1
                begin += 1
            # ------- End ------ #
            # now [begin, end) is the first invalid window waiting
            # to be updated in the next iteration
        
        return ans

### 78 Subsets
- Backtracking中不要忘记 list 需要 copy, 不然 accumulate 的是同一个 list 的 reference
- Copy a list in python using `lst.copy()`

In [ ]:
class Solution:
    def subsets(self, nums: List[int]) -> List[List[int]]:
        ####################################
        # Solution 1 using recursion
        ####################################
        # Base case
        if len(nums) == 0:
            return []
        if len(nums) == 1:
            return [[], nums]
        # Recursive case
        rec_res = self.subsets(nums[1:])
        full_res = []
        for subset in rec_res:
            full_res.append(subset)
            full_res.append(subset + [nums[0]])
        return full_res
    
        ####################################
        # Solution 2 using backtrack
        ####################################
        # The nums are N1, N2, N3, ..., Nn
        all_sets = []
        def backtrack(cursor, cur_set):
            # base case: always arriving at a YES leaf
            all_sets.append(cur_set[:])
            # recursive case
            for i in range(cursor, len(nums)):
                cur_set.append(nums[i])
                backtrack(cursor, cur_set)
                cur_set.pop()
            

### 79 Word Search
- 本质上是 `backtracking`
- graph 上的 backtracking 可以参考本题中 apply change `board[r][c] = !` 和 `board[r][c] = old` 的做法
- Destructively change the board to mark a cell as `visited`, 但是check要放在`with_range(row, col)` 之后
```
ans = False
def backtrack(row, col, remain):
    # base case
    if len(remain) == 0:
        ans = True
        return
    if row < 0 or row >= row_n or col < 0 or col >= col_n:
        return
    if board[row][col] == '!'：
        return

    # recursive case
    (1) Apply change
    old_symbol = board[row][col]
    board[row][col] = '!'

    (2) backtrack
    for (d_row, d_col) in ... : 
        new_row, new_col = row+d_row, col+d_col
        backtrack(new_row, new_col, remain[1:])

    (3) restore change
    board[row][col] = old_symbol
    

```

In [ ]:
class Solution(object):      
    def exist(self, board, word):
        """
        :type board: List[List[str]]
        :type word: str
        :rtype: bool
        """
        rowN = len(board)
        colN = len(board[0])
        
        # a BFS helper function
        # 本质上是backtrack
        def BFS(row, col, target):
            # Base Case
            if len(target) == 0:
                return True
            if row < 0 or row >= rowN or col < 0 or col >= colN or board[row][col] != target[0]:
                return False
            if board[row][col] == '!':
                return False
             
            # Recursive Case
            success = False
            old = board[row][col]
            board[row][col] = '!'
            
            for (dRow, dCol) in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                newRow = row + dRow
                newCol = col + dCol
                success = success or BFS(newRow, newCol, target[1:])
            
            # revert
            board[row][col] = old
            return success
        
        # Main
        for r in range(rowN):
            for c in range(colN):
                if BFS(r, c, word):
                    return True
        return False

### 79 Word Search
- Don't forget to use `backtrack` algorithm
- Backtracking特点 <br>
1 Have a target to search for or need to accumulate all sols <br>
2 Need to potentially give up the current move 

In [ ]:
class Solution(object):      
    def exist(self, board, word):
        """
        :type board: List[List[str]]
        :type word: str
        :rtype: bool
        """
        rowN = len(board)
        colN = len(board[0])
        
        # a BFS helper function
        def backtrack(row, col, target):
            # Base Case
            if len(target) == 0:
                return True
            if row < 0 or row >= rowN or col < 0 or col >= colN or board[row][col] != target[0]:
                return False
            if board[row][col] == '!': # visited
                return False
        
            # Recursive Case
            success = False
            # (1) apply change
            old = board[row][col]
            board[row][col] = '!'
            # (2) recursively backtrack
            for (dRow, dCol) in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                newRow = row + dRow
                newCol = col + dCol
                success = success or backtrack(newRow, newCol, target[1:])
            # (3) revert change
            board[row][col] = old
            return success
        
        # Main
        for r in range(rowN):
            for c in range(colN):
                if backtrack(r, c, word):
                    return True
        return False
                    
        

### 81 Search in Rotated Sorted Array II
 - See `imgs/81-1.png`
 - See `imgs/81-2.png`

In [ ]:
def search(self, nums: List[int], target: int) -> bool:
        lo, hi = 0, len(nums)
        
        while lo < hi:
  
            mid = (lo + hi) // 2
            midNum = nums[mid]
            startNum = nums[0]
            # --- Found
            if midNum == target or startNum == target: return True
            
            # --- Not found immediately
            # --- --- Case 1: The mid is in the left triangle
            if midNum > startNum:
                if target > midNum:
                    lo = mid + 1
                elif target > startNum:
                    hi = mid
                else: # target < startNum
                    lo = mid + 1
            # --- --- Case 2: The mid is in the right triangle
            elif midNum < startNum:
                if target > startNum:
                    hi = mid
                elif target > midNum:
                    lo = mid + 1
                else: # target < midNum
                    hi = mid
            # --- --- Case 3: We cannot tell
            else:
                return target in nums[lo : hi+1]
        print(lo, hi)
        return False

### 82 Remove Duplicates from Sorted List II
 - Use `dummy` head and tail
 - $\textit{Three Pointer Scan}$ -- `prev, curr, succ` with `while succ and  (prev.val == curr.val or curr.val == succ.val):` to find the next distinct number
 - Finally remove the dummy head and tail

In [ ]:
def deleteDuplicates(self, head: Optional[ListNode]) -> Optional[ListNode]:
    # 1 -- Edge cases
    if not head or not head.next: return head
    
    # 2 -- Build dummy head and dummy tail
    dummy_head = ListNode(math.inf, head)
    prev, curr = None, head
    while curr:
        prev = curr
        curr = curr.next
    dummy_tail =  ListNode(math.inf, None)
    prev.next = dummy_tail
    
    
    prev, curr, succ = dummy_head, head, head.next
    while curr:
        last_distinct = prev
        while succ and  (prev.val == curr.val or curr.val == succ.val):
            # move 
            prev = curr
            curr = succ
            succ = succ.next
        last_distinct.next = curr
        # move
        prev = curr
        curr = succ
        succ = succ.next if succ else None
    
    # 3 -- get rid of dummy tail
    prev, curr = dummy_head, dummy_head.next
    while curr != dummy_tail:
        prev = curr
        curr = curr.next
    prev.next = None
    return dummy_head.next

### 83 Remove Duplicates from Sorted List
 - Making heap inside iteration `linked list problem: while loop inside while loop` 的时候
    - 1. 注意handle degenerate case 
         ```
            if head is None or head.next is None:return head
         ```
    - 注: 假设`last` is the leading cursor in the linked list
    - 2. 外部 while loop,
         ```
            while last is not None: ---
         ```
    - 3. 内部 while loop,
         ```
            while last and [extra condition]: ---
         ```
    - 4. 内部 while loop 有可能根本不执行, 内部loop之后固定的move cursor for once
         ```
           prev = curr
           curr = curr.next is curr else None # 不要忘记check
         ```

In [ ]:
def deleteDuplicates(self, head: Optional[ListNode]) -> Optional[ListNode]:
    # 1 -- handle degenerate case
    if head is None or head.next is None:return head
    
    # 2 -- normal cases
    curr, succ = head, head.next
    while succ:
        last_distinct = curr
        while succ and curr.val == succ.val:
            curr = succ
            succ = succ.next
        last_distinct.next = succ
        curr = succ
        succ = succ.next if succ else None
    return head

### 86 Partition List
 - collect greater and less nodes into two links separately and connect the two links together at last

In [ ]:
def partition(self, head: Optional[ListNode], x: int) -> Optional[ListNode]:
    dummy_less = ListNode(-1)
    dummy_greater = ListNode(-2)
    dummy = ListNode(-3, head)
    
    cursor_less = dummy_less
    cursor_greater = dummy_greater
    cursor = head
    
    # traverse the linked list
    while dummy.next:
        # 1-- remove the node from the 
        node = dummy.next
        dummy.next = dummy.next.next
        
        # 2-- install the node to where it belongs to
        node.next = None
        if node.val < x:
            cursor_less.next = node
            cursor_less = node
        else:
            cursor_greater.next = node
            cursor_greater = node
            
    # connect the two linked lists into one
    cursor_less.next = dummy_greater.next
    return dummy_less.next

### 87 Scramble String
 - Performance $O(m^2n^2)$ with memoization
 - Dynamic Programming: __Optimal Binary Search Tree Model__ + __every branch test (l, r) and (r, l)__
 - Subproblems `i1, j1, i2, j2 -> whether s1[i1:j1] is a scrambled string of s2[i2:j2]`

In [ ]:
def isScramble(self, s1: str, s2: str) -> bool:
    # Subproblem i1, j1, i2, j2 -> whether s1[i1:j1] is a scrambled string of s2[i2:j2]
    # There are at most M^2 N^2 subproblems
    cache = dict()
    
    # whether s1[i1:j1] is a scrambled string of s2[i2:j2]
    def DP(i1, j1, i2, j2):
        #### Base cases
        if (i1, j1, i2, j2) in cache:
            return cache[(i1, j1, i2, j2)]
        if j1 - i1 == 1 and j2 - i2 == 1:
            cache[(i1, j1, i2, j2)] = s1[i1] == s2[i2]
            return  s1[i1] == s2[i2]
        if j1 - i1 <= 0 and j2 - i2 <= 0:
            cache[(i1, j1, i2, j2)] = True
            return True
        if j1 - i1 <= 0 or j2 - i2 <= 0:
            cache[(i1, j1, i2, j2)] = False
            return False
        if j1 - i1 != j2 - i2:
            cache[(i1, j1, i2, j2)] = False
            return False
        
        #### Recursive case
        recs = False
        for l in range(1, j1-i1):
            noSwap = DP(i1, i1+l, i2, i2+l) and DP(i1+l, j1, i2+l, j2)
            yesSwap = DP(i1, i1+l, j2-l, j2) and DP(i1+l, j1, i2, j2-l)
            recs = recs or noSwap or yesSwap
        cache[(i1, j1, i2, j2)] = recs
        return recs

    # Driver
    res = DP(0, len(s1), 0, len(s2))
    return res

### 89 Gray Code
 - `_ ^ 1` does not change `_`
 - `_ ^ 0` flips `_`
 ```
 for i in range(n):
    num = sol[-1] ^ (1 << i)
 ```

In [ ]:
def grayCode(self, n: int) -> List[int]:
    # -- sol: a list keeping track of the current solution
    # -- used: a set keeping track of all the numbers
    ans = None
    def backtrack(sol, used):
        nonlocal ans
        if len(sol) == 2 ** n:
            ans = sol.copy()
            return True
        else:
            for i in range(n):
                num = sol[-1] ^ (1 << i)
                if num in used: 
                    continue

                # -- apply change
                sol.append(num)
                used.add(num)
                # -- recurse
                if backtrack(sol, used):
                    return True
                # -- recall change
                sol.pop()
                used.remove(num)
            return False

    # -- Driver
    backtrack([0], {0})

### 90 Subsets II
- identical to _40. Combination Sum II_ to avoid redundant subsets

In [ ]:
class Solution:
    def subsetsWithDup(self, nums: List[int]) -> List[List[int]]:
        nums.sort()
        n = len(nums)
        subsets = []
        
        def backtrack(cursor, subset):
            subsets.append(subset.copy())

            for new_cursor in range(cursor, n):
                new_elem = nums[new_cursor]
                if new_cursor > cursor and new_elem == nums[new_cursor - 1]:
                    continue
                subset.append(new_elem)
                backtrack(new_cursor+1, subset)
                subset.pop()
            
        backtrack(0, [])
        return subsets

### 91 Decode Ways
- Every time look back by 2 elems, so init 2 bases
- Every  time get `x` and `y`
- Out case on `y == 0` to see if can be parsed as `x, y` $\implies$ DP[l-1]
- Inner case on `10 <= xy <= 26` to see if can be parsed as `xy`$\implies$ DP[l-2]

In [ ]:
class Solution(object):
    def numDecodings(self, s):
        """
        :type s: str
        :rtype: int
        DP[l] = the number of encodings for the l-prefix of s
        """
        DP = [0] * (len(s) + 2)
        nums = [0] * (len(s) + 2)
        for i in range(len(s)):
            nums[i] = ord(s[i]) - ord('1') + 1
        
        # Base Case
        # DP[0]
        DP[0] = 0
        # DP[1]
        if nums[0] == 0:
            DP[1] = 0
        else:
            DP[1] = 1
        # DP[2]
        if nums[0] == 0:
            DP[2] = 0
        else:
            n = 10 * nums[0] + nums[1]
            if n <= 26:
                DP[2] += 1 # can be parsed as xy 
            if nums[0] != 0 and nums[1] != 0:
                DP[2] += 1 # can be parsed as x, y
        
        # Recursive Case
        for l in range(3, len(s)+1):
            x = nums[l-2]
            y = nums[l-1]
            xy = 10*x + y
            # Outer: case on y
            # Inner: case on xy
            if y != 0:
                DP[l] += DP[l-1] # can parse as x, y
                if 11 <= xy <= 26: # can parse xy
                    DP[l] += DP[l-2]
            elif y == 0: # cannot parse as x, y
                if x == 1 or x == 2: # can parse as xy
                    DP[l] += DP[l-2]
        return DP[len(s)]       

### 92 Reverse Linked List II

In [ ]:
def reverseBetween(self, head: Optional[ListNode], left: int, right: int) -> Optional[ListNode]:
    # handle degenerate case
    if not head: return None
    
    # handle normal cases
    # 1-- set the start sentinel
    dummy = ListNode(-1, head)
    start_prev, start = None, dummy
    for _ in range(left): 
        start_prev = start
        start = start.next
    
    # 2-- set the end sentinel
    end, end_post = dummy, dummy.next
    for _ in range(right): 
        end = end.next
        end_post = end_post.next
    
    # 3-- reverse the part from [left, right]
    prev, curr = start, start.next
    while prev != end:
        succ = curr.next
        curr.next = prev
        prev = curr
        curr = succ
    
    # 4-- install the reversed parts as a whole back to the linked list
    start_prev.next = end
    start.next = end_post
    return dummy.next

### 93 Restore IP Addresses
 - Encoding each way of dividing an IP using a list of all the lengths of each part

In [ ]:
def restoreIpAddresses(self, s: str) -> List[str]:
    all_valid_divisions = []
    n = len(s)
    
    # trying to parse s[i:] using dotN dots to make a valid ip address
    def backtrack(i, dotN, allLens):
        # print(i, dotN, allLens)
        # -- Base case
        if i >= n:
            if dotN == 0:
                all_valid_divisions.append(allLens.copy())
            return
        
        # -- recursive case
        for thisLen in [1,2,3]:
            if i + thisLen > n: continue
            string = s[i : i+thisLen]
            leadingZero = string[0] == '0' and thisLen > 1
            if not leadingZero and 0 <= int(string) <= 255:
                allLens.append(thisLen)
                backtrack(i+thisLen, dotN-1, allLens)
                allLens.pop()
    # Driver
    backtrack(0, 4, [])
    # Translate 'all part lens' to explicit IPs
    IPs = []
    for lens in all_valid_divisions:
        i = 0
        chunks = []
        for l in lens:
            chunks.append(s[i:i+l])
            i = i + l
        IP = '.'.join(chunks)
        IPs.append(IP)
    return IPs

### 94 Binary Tree Inorder Traversal
 - See pseudo for `iterative tree inorder traversal` below

In [ ]:
def inorderTraversal(self, root: Optional[TreeNode]) -> List[int]:
    vals = []
    stack = []
    def goLeftFar(node):
        while node:
            stack.append(node)
            node = node.left
            
    # -- init
    goLeftFar(root)

    # -- start traversing
    while stack:
        topNode = stack.pop()
        vals.append(topNode.val)
        if topNode.right: goLeftFar(topNode.right)
    return vals

### 95 Unique Binary Search Trees II
 - Dynamic Programming 
    - Optimal Binary Search Tree Structure
    - Loop through all possible `partitin points` (roots)

In [ ]:
def generateTrees(self, n: int) -> List[Optional[TreeNode]]:
    # n -> all the trees with n nodes 1 ... n
    cache = dict() 
    
    # all the trees with n nodes x ... y both inclusive
    def DP(x, y):
        #### Base cases
        if x > y: 
            cache[(x, y)] = [None]
            return [None]
        if x == y:
            cache[(x, y)] = [TreeNode(x)]
            return cache[(x, y)]
        
        #### Recursive cases
        else: # x < y
            trees = []
            for r in range(x, y+1):
                # --            r
                # --           /  \
                # --    (x, r-1)   (r+1, y)
                leftSubs = DP(x, r-1)
                rightSubs = DP(r+1, y)
                for L in leftSubs:
                    for R in rightSubs:
                        M = TreeNode(r)
                        M.left = L
                        M.right = R
                        trees.append(M)
            cache[(x, y)] = trees
            return trees
    
    # Driver
    allTrees = DP(1, n)
    return allTrees

### 96 Unique Binary Search Trees
 - Dynamic Programming

In [ ]:
def numTrees(self, n: int) -> int:
    cache = dict()
    
    def DP(i):
        #### Base case
        if i in cache:
            return cache[i]
        if i == 0 or i == 1: 
            cache[i] = 1
            return 1
        
        #### Recursive case
        else:
            treeNum = 0
            for leftSize in range(0, i):
                rightSize = i - 1 - leftSize
                treeNum += DP(leftSize) * DP(rightSize)
            cache[i] = treeNum
            return treeNum
    
    # Driver
    ans = DP(n)
    return ans

### 97 Interleaving String
 - Sol 1: recursion with memoization forward
    - 类似于从左到右parse string的可以将DP variable 设置成为cursor
 - Sol 2: DAG traversal
    - recurrence $$DP[i][j] = (s1[i-1] == s3[i+j-1] \land DP[i-1][j]) \lor (match2 = s2[j-1] == s3[i+j-1] \land DP[i][j-1])$$

In [ ]:
def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
        cache = dict()
        
        # if s1[i:] and s2[j:] constitutes an interleaving for s3[i+j:]
        def DP(i, j):
            #### Base case
            if (i, j) in cache: 
                return cache[(i, j)]
            if i + j >= len(s3):
                cache[(i, j)] = False
                return False
            if i >= len(s1) and j >= len(s2):
                if i+j >= len(s3):
                    cache[(i, j)] = True
                    return True
                else:
                    cache[(i, j)] = False
                    return False
                
            #### Recursive case
            else:
                match1, match2 = False, False
                if i < len(s1):
                    match1 = s1[i] == s3[i+j] and DP(i+1, j)
                    
                if j < len(s2):
                    match2 = s2[j] == s3[i+j] and DP(i, j+1)
                cache[(i, j)] = match1 or match2
                return cache[(i, j)]
        
        # Driver
        ans = DP(0, 0)
        return ans


def isInterleave(self, s1: str, s2: str, s3: str) -> bool:
    # Handle degenerate case
    if len(s1) + len(s2) != len(s3):
        return False
    if len(s1) == 0: return s2 == s3
    if len(s2) == 0: return s1 == s3
    if len(s1) ==0 and len(s2) == 0: return True
    
    # Handle normal cases
    # -- DP[i][j] = if s1[:i] and s2[:j] interleaves to be s3[i+j:]
    m = len(s1)
    n = len(s2)
    DP = self.createMatrix(m+1, n+1, False)
    
    # Base case
    DP[0][0] = True # "" + "" = "" trivially holds
    #-- Only using s1
    for i in range(1, m+1):
        DP[i][0] = DP[i-1][0] and s1[i-1] == s3[i-1]
    #-- Not using s1
    for j in range(1, n+1):
        DP[0][j] = DP[0][j-1] and s2[j-1] == s3[j-1]
    
    # Recursive case
    #  * * * * * * * * *
    for i in range(1, m+1):
        for j in range(1, n+1):
            match1 = s1[i-1] == s3[i+j-1] and DP[i-1][j]
            match2 = s2[j-1] == s3[i+j-1] and DP[i][j-1]
            DP[i][j] = match1 or match2
            
    return DP[m][n]
        

### 98 Validate Binary Search Tree

In [ ]:
# Definition for a binary tree node.
# class TreeNode(object):
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution(object):
def inOrder(self, root):
    # Base Case
    if root is None:
        return []
    # Recursive Case
    return self.inOrder(root.left) + [root.val] + self.inOrder(root.right)
    
def isSorted(self, l):
    n = len(l)
    for i in range(n-1):
        if l[i] >= l[i+1]:
            return False
    return True

def checkWithRange(self, root, lo, hi):
    # Base Case
    if root is None:
        return True
    
    # Recursive Case
    # check current node
    cur = root.val
    if lo is not None and lo >= cur:
        return False
    if hi is not None and hi <= cur:
        return False
    # check left and right children
    return self.checkWithRange(root.left, lo, cur) and self.checkWithRange(root.right, cur, hi)

def isValidBST(self, root):
    """
    :type root: TreeNode
    :rtype: bool
    """
    '''
    linear = self.inOrder(root)
    return self.isSorted(linear)
    '''
    return self.checkWithRange(root, None, None)
    
    

### 99 Recover Binary Search Tree
 - Method 1: Construct the in order traversal -> detect the two elements that are swaped -> traverse the tree again and change the value `x` to `y` and `y` to `x`.
    - to detect `x` -> `nums[x-1] < nums[x] > nums[x+1]`
    - to detect `y` -> `nums[y-1] > nums[y] < nums[y+1]`
    

In [ ]:
def recoverTree(self, root: Optional[TreeNode]) -> None:
    # 1-- Genertate the inorder traversal and detect the values swapped
    nums = []
    def linearize(root):
        if root is None:
            return
        else:
            linearize(root.left)
            nums.append(root.val)
            linearize(root.right)
    linearize(root)
    x, y = None, None
    for i in range(len(nums)):
        left = nums[i-1] if i > 0 else -math.inf
        right = nums[i+1] if i < len(nums)-1 else math.inf
        if not (left < nums[i] < right):
            if x is None: x = nums[i]
            else: y = nums[i]
        
    # 2-- Travers the tree again and reassigne values x and y
    stack = [root]
    while stack:
        node = stack.pop()
        if node.val == x: node.val = y
        elif node.val == y: node.val = x 
        if node.left: stack.append(node.left)
        if node.right: stack.append(node.right)

### 100 Same Tree

In [ ]:
class Solution(object):
    def isSameTree(self, p, q):
        """
        :type p: TreeNode
        :type q: TreeNode
        :rtype: bool
        """
        # Base Case
        if p is None and q is None:
            return True
        if p is None or q is None:
            return False
        # now we know that both p and q are not None
        return p.val == q.val and self.isSameTree(p.left, q.left) and self.isSameTree(p.right, q.right)

### 0 All graph-related pseudocode
 - DFS 
    - recursive
        ```
        dfs(v):
            if v is visited:
                return
            else:
                mark v as visited
                for u in G.adj[v]:
                    dfs(u)
        ```
    - iterative
        ```
        def DFS(v):           
            visited = [False for i in range(V)]
            stack = [v]
 
            while stack:
                s = stack.pop()
                visit(s)
                visited[s] = True
                for node in adj[s]:
                    if (not visited[node]):
                        stack.append(node)
        ```

 - BFS
    ```
    from collections import deque
    def bfs(G, s):
        Q = deque([s])
        visited[s] = true
        while Q:
            v = Q.popleft()
            do(v)
            for u in G.adj[v]:
                if visited[u] == false:
                    Q.append(v)
                    visited[v] = true  
    ```
 
 - Topological sort + Cycle detection pseudo code
    -  Global variables
        ```
        # topological sort on the graph
        sorted = ""
        visited = set()
        ```
    - Topo sort using DFS algorithm
        ```
        # detecing for cycle while accumulating the topo sort
        def topoSort_hasCycle(v, stack):
            nonlocal sorted
            if v in stack: return True
            if v in visited: return False
            
            visited.add(v)
            stack.append(v)
            for u in adj[v]:
                if topoSort_hasCycle(u, stack): return True
            sorted = v + sorted
            stack.pop()
            return False
        # driver
        for v in adj:
            stack = []
            if topoSort_hasCycle(v, stack): return __
        ```
    - Topo sort using indegree algorithm
        ```
        def topoSort_indegree(v):
            leaves = deque([c for c in in_degree if in_degree[c] == 0])
            while leaves:
                leaveSize = len(leaves)
                for _ in range(leaveSize):
                leaf = leaves.popleft()
                output.append(leaf)
                for nbor in adj[leaf]:
                    in_degree[leaf] -= 1
                    if in_degree[leaf] == 0:
                        leaves.append(leaf)
        # Driver
        compare sorted to len(in_degree) to see if all vertices are eliminated to 0 indegree
        ```

### 0 Tree traversal
 - Iterative PreOrder Traversal
 - Iterative PostOrder Traversal
 - Iterative InOrder Traversal
    - 1. Pop the top node from the stack
    - 2. If the top node has right child, collect nodes along the left path of the top node
    ```
        def iterInorder(root):
            stack = []
            def goLeftFar(node):
                while node:
                    stack.append(node)
                    node = node.left
            # -- init
            goLeftFar(root)

            # -- start traversing
            while stack:
                topNode = stack.pop()
                print(topNode.val)
                if topNode.right: goLeftFar(topNode.right)
        
    ```
 - Preorder Traversal tracking prev
 - Postorder Traversal tracking prev
 - InOrder Traversal tracking prev
    ```
    def inorder(self, root, prev):
        # Base case
        if root.left is None and root.right is None:
            self.display(prev)
            self.display(root)
            return root
        
        # Recursive case
        # 1-- Left
        lastLeft = None
        if root.left:
            lastLeft = self.inorder(root.left, prev)
        
        # 2-- Mid
        self.display(lastLeft)
        self.display(root)
        
        # 3-- Right
        lastRight = None
        if root.right:
            lastRight = self.inorder(root.right, root)
            
        return lastRight if lastRight else root
    ```